#                Checkpoint 9 Calculation

<div>
<img src="checkpoint9_tf1.jpg" width="800"/>
</div>

<div>
<img src="checkpoint9_tf2.jpg" width="800"/>
</div>

<div>
<img src="checkpoint9_tf3.jpg" width="800"/>
</div>

<div>
<img src="checkpoint9_tf4.jpg" width="800"/>
</div>

<div>
<img src="checkpoint9_tf5.jpg" width="800"/>
</div>

<div>
<img src="checkpoint9_tf6.jpg" width="800"/>
</div>

# Explanation
position = (x,y) position
yet = to be known by calcuation
o is at origin of x,y plane

## unknown yet to be found
- l1, l2 positions are unknown, yet 
- p is the mid-point along the line l1-l2, thus unknown, yet
- alpha1

## known value
- theta1, theta2 
- alpha2
- b,c length
- a length (from origin of robot to laser)
- r radius of the robot 

## trivial derived known value
- l1-l2 length derives from b,c
- l1-p, l2-p length derive from l1-l2 length
- theta3 derive from theta2
- o-l1 derive from a,b, theta3 (cosine law)
- laser position

## nontrivial derived known value
- let d be projection of laser scanner point to line o-l1
we can solve 4 equations with 4 unknown by method of substitution, and bisection
to find theta31, theta32
- alpha3 can be known from theta32
- thus l1 can be known by polar coordinate, calculate back to position in cartesian
- thus l2, can follow l1 process and find the position
- thus p position can be derive from l1,l2 position

In [1]:
import numpy as np
import math

# answer
l1_ans = np.array([8,6], dtype='f')
l2_ans = np.array([10,-2], dtype='f')
a = 1
alpha2_l1 = math.radians(10)
alpha3_add_alpha2_l1_ans = math.atan(l1_ans[1]/l1_ans[0])
print(math.degrees(alpha3_add_alpha2_l1_ans))

# calculate known value
import math


alpha4_l2_ans = math.atan(l2_ans[1]/l2_ans[0])
print('alpha4_l2_ans',alpha4_l2_ans, math.degrees(alpha4_l2_ans))
alpha3_l1_ans = alpha3_add_alpha2_l1_ans - alpha2_l1
print('alpha3_l1_ans',alpha3_l1_ans)
theta_32_l1_ans = math.radians(90) - alpha3_l1_ans 
laser = np.array([a* math.cos(alpha2_l1),a*math.sin(alpha2_l1)] ,dtype ='f')
print('laser',laser)
theta1_l2 = math.atan((l2_ans[1] - laser[1])/(l2_ans[0]-laser[0]))- alpha2_l1
l2_laser_diff_ans = l2_ans - laser
theta1_l2 = math.acos(np.dot(l2_laser_diff_ans, laser)/(np.linalg.norm(l2_laser_diff_ans)*np.linalg.norm(laser)))
print('theta1_l2',theta1_l2)
theta2_l1 = math.atan((l1_ans[1] - laser[1])/(l1_ans[0]-laser[0]))- alpha2_l1
print('theta2_l1 ',theta2_l1 )
theta3_l1 = math.radians(180) - theta2_l1
theta4_l2 = math.radians(180) - theta1_l2
print('theta3_l1', theta3_l1, math.degrees(theta3_l1))
print('theta4_l2', theta4_l2, math.degrees(theta4_l2))
b = math.sqrt((l1_ans[1] - laser[1])**2+(l1_ans[0] - laser[0])**2)
c = math.sqrt((l2_ans[1] - laser[1])**2+(l2_ans[0] - laser[0])**2)
print('b,c',b,c)
ol1 = math.sqrt(a**2+b**2-2*a*b*math.cos(theta3_l1))
ol1_ans = math.sqrt(l1_ans[0]**2+l1_ans[1]**2)
ol2 = math.sqrt(a**2+c**2-2*a*c*math.cos(theta4_l2))
ol2_ans = math.sqrt(l2_ans[0]**2+l2_ans[1]**2)
d_ans = l1_ans*np.dot(l1_ans,laser)/ol1_ans**2
l1d_ans = math.sqrt((l1_ans[0] - d_ans[0])**2 + (l1_ans[1] - d_ans[1])**2)
theta31_ans = math.asin(l1d_ans/b)
print(laser,b)
print(math.degrees(theta2_l1))
print('ol1',ol1,ol1_ans)
print('ol2',ol2,ol2_ans)
print(math.degrees(theta31_ans))

In [2]:
def f(x,a,b,theta3,ol1):
    return b*math.sin(x)+a*math.sin(theta3 - x) - ol1

In [3]:
def fprime(x,a,b,theta3,ol1):
    return b*math.cos(x) - a*math.cos(theta3 - x)

print(f(math.radians(87.15918799293077),a,b,theta3_l1,ol1))
print(f(math.radians(-10),a,b,theta3_l1,ol1))

def newton_method(x,a,b,theta3,ol1):
    val = f(x,a,b,theta3,ol1)
    while(abs(val)>0.0000001):
        x = x - f(x,a,b,theta3,ol1)/fprime(x,a,b,theta3,ol1)
        val =  f(x,a,b,theta3,ol1)
    return x,val

# Test newton_method
x_test_newton_method, val_test_newton_method = newton_method(math.radians(30),a,b,theta3_l1,ol1)
print(math.degrees(x_test_newton_method),val_test_newton_method )

# Calculate theta31_l1 with newton method
theta31_l1,_  = newton_method(math.radians(30),a,b,theta3_l1,ol1)
theta32_l1 = theta3_l1 - theta31_l1
alpha3_l1 = math.radians(90) - theta32_l1 
print(theta31_l1)
print(alpha3_l1)

# Calculate theta42_l2 with newton method
theta41_l2,_  = newton_method(math.radians(30),a,c,theta4_l2,ol2)
theta42_l2 = theta4_l2 - theta41_l2
alpha4_l2 = -1*(math.radians(90) - theta42_l2  - alpha2_l1 )
print(theta42_l2 )
print(alpha4_l2)

# Calculate theta31_l1 with law of cosine
# sin(alpha3_l1)/b = sin(theta3)/ol1
# alpha3_l1= arc_sin(b/ol1*sin(theta3))
alpha3_l1 = math.asin(b/ol1*math.sin(theta3_l1))
print(alpha3_l1)

# Calculate theta42_l2 with law of cosine
# sin(alpha4_l2)/a = sin(theta4)/ol2
# alpha4_l2_add_alpha2 = arc_sin(c/ol2*sin(theta4))
alpha4_l2_add_alpha2 = math.asin(c/ol2*math.sin(theta4_l2))
alpha4_l2 =-(alpha4_l2_add_alpha2 - alpha2_l1)
print(alpha4_l2)

l1y = ol1*math.sin(alpha2_l1+alpha3_l1)
l1x = ol1*math.cos(alpha2_l1+alpha3_l1)
l1 = np.array([l1x,l1y])
print(l1x,l1y,l1)

def find_leg_position(a,b,c,theta1,theta2,alpha2):
    laser = np.array([a* math.cos(alpha2),a*math.sin(alpha2)] ,dtype ='f')
    theta3 = math.radians(180) - theta2
    theta4 = math.radians(180) - theta1
    print('theta3',math.degrees(theta3))
    print('theta4',math.degrees(theta4))
    ol1 = math.sqrt(a**2+b**2-2*a*b*math.cos(theta3))
    ol2 = math.sqrt(a**2+c**2-2*a*c*math.cos(theta4)) 
    print('ol2',ol2)
    alpha3 = math.asin(b/ol1*math.sin(theta3))
    print('alpha3',alpha3)
    alpha4_plus_alpha2 = math.asin(c/ol2*math.sin(theta4))
    print('alpha4_plus_alpha2',alpha4_plus_alpha2)
    alpha4 =   alpha2 -alpha4_plus_alpha2
    print('alpha4',alpha4, math.degrees(alpha4))
    l1y = ol1*math.sin(alpha2+alpha3)
    l1x = ol1*math.cos(alpha2+alpha3)
    l2y = ol2*math.sin(alpha4)
    l2x = ol2*math.cos(alpha4)
    return np.array([l1x,l1y]), np.array([l2x,l2y])

print(a,b,c,theta1_l2,theta2_l1,alpha2_l1)
leg1, leg2 = find_leg_position(a,b,c,theta1_l2,theta2_l1,alpha2_l1)
print(leg1,leg2)


In [4]:
l1_ans = np.array([6,8], dtype='f')  
l2_ans = np.array([10,-2], dtype='f')
theta_l1l2_ans =  math.acos(np.dot(l1_ans,l2_ans)/(np.linalg.norm(l1_ans)*np.linalg.norm(l2_ans)))
print(math.degrees(theta_l1l2_ans))
assert(math.radians(180) >abs(theta_l1l2_ans) > math.radians(20))
assert(l1_ans[1]>l2_ans[1])
a = 5
alpha2 = math.radians(10)
print('alpha2',alpha2)
laser = np.array([a* math.cos(alpha2),a*math.sin(alpha2)] ,dtype ='f')
print('laser',laser)
b = math.sqrt((l1_ans[1] - laser[1])**2+(l1_ans[0] - laser[0])**2)
c = math.sqrt((l2_ans[1] - laser[1])**2+(l2_ans[0] - laser[0])**2)
print('b',b)
print('c',c)
l2_laser_diff_ans = l2_ans - laser
l1_laser_diff_ans = l1_ans - laser
linear_algebra2 = np.dot(l2_laser_diff_ans, laser)/(np.linalg.norm(l2_laser_diff_ans)*np.linalg.norm(laser))
theta2 = -math.acos(linear_algebra2)
linear_algebra1 = np.dot(l1_laser_diff_ans, laser)/(np.linalg.norm(l1_laser_diff_ans)*np.linalg.norm(laser))
theta1 = math.acos(linear_algebra1)
print(linear_algebra1, linear_algebra2)

assert(1-abs(linear_algebra1)>0.1)
assert(1-abs(linear_algebra2)>0.1)
print('theta1',theta1)
print('theta2',theta2)
print(math.degrees(theta1),math.degrees(theta2))

64.44003510322344
alpha2 0.17453292519943295
laser [4.924039  0.8682409]
b 7.21246706257271
c 5.830281871243811
0.31861937 0.77196634
theta1 1.24652373718018
theta2 -0.6888675951149327
71.42054920329898 -39.469205843412446


In [5]:
laser_to_l1 = np.array([b*math.cos(alpha2+theta1),b*math.sin(alpha2+theta1)])
laser_to_l2 = np.array([c*math.cos(alpha2+theta2),c*math.sin(alpha2+theta2)])
l1 = laser + laser_to_l1
l2 = laser + laser_to_l2
print(l1,l2)

[5.99999988 8.00000008] [10.00000022 -1.99999955]


In [6]:
RotationMatrix = np.array([[math.cos(alpha2), -math.sin(alpha2)],[math.sin(alpha2),math.cos(alpha2)]],dtype='f')
print(RotationMatrix)
l1_laser_coordinate = b*np.array([math.cos(theta1), math.sin(theta1)], dtype='f')
l2_laser_coordinate = c*np.array([math.cos(theta2), math.sin(theta2)], dtype='f')
lmid_laser_coordinate = (l1_laser_coordinate+l2_laser_coordinate )/2
print(l1_laser_coordinate, l2_laser_coordinate,lmid_laser_coordinate)

[[ 0.9848077  -0.17364818]
 [ 0.17364818  0.9848077 ]]
[2.2980318 6.8365736] [ 4.500781  -3.7060966] [3.3994064 1.5652385]


In [7]:
l1_o = np.dot(RotationMatrix, l1_laser_coordinate) + laser
l2_o = np.dot( RotationMatrix, l2_laser_coordinate) + laser;
lmid_o =  np.dot( RotationMatrix, lmid_laser_coordinate) + laser;
print(l1_o,l2_o,lmid_o)

[6.       8.000001] [10.        -1.9999995] [8.        3.0000005]


## Finding a perpendiculat vector to P1-P2 in Laser Coordinate

<div>
<img src="cart_frame_direction1.jpg" width="800"/>
</div>

<div>
<img src="cart_frame_direction2.jpg" width="800"/>
</div>

In [8]:
l1_to_l2_laser_coordinate = l2_laser_coordinate - l1_laser_coordinate
print(l1_to_l2_laser_coordinate)

[  2.2027493 -10.54267  ]


In [9]:
perpen_l1_to_l2_laser_coordinate_x__1 = 1
perpen_l1_to_l2_laser_coordinate_y__1 = -l1_to_l2_laser_coordinate[0]/l1_to_l2_laser_coordinate[1]
perpen_l1_to_l2_laser_coordinate__1 = np.array([perpen_l1_to_l2_laser_coordinate_x__1 ,perpen_l1_to_l2_laser_coordinate_y__1])
print(perpen_l1_to_l2_laser_coordinate__1 )
print(np.dot(perpen_l1_to_l2_laser_coordinate__1, l1_to_l2_laser_coordinate ))

[1.         0.20893656]
2.6879192205342406e-08


In [10]:
perpen_l1_to_l2_laser_coordinate_x__2 = -1
perpen_l1_to_l2_laser_coordinate_y__2 = l1_to_l2_laser_coordinate[0]/l1_to_l2_laser_coordinate[1]
perpen_l1_to_l2_laser_coordinate__2 = np.array([perpen_l1_to_l2_laser_coordinate_x__2 ,perpen_l1_to_l2_laser_coordinate_y__2])
print(perpen_l1_to_l2_laser_coordinate__2 )
print(np.dot(perpen_l1_to_l2_laser_coordinate__2, l1_to_l2_laser_coordinate ))

[-1.         -0.20893656]
-2.6879192205342406e-08


In [11]:
yaw_degree_perpendicular_Pmid = math.acos(np.dot(perpen_l1_to_l2_laser_coordinate__1,lmid_laser_coordinate)\
                                          /(np.linalg.norm(perpen_l1_to_l2_laser_coordinate__1)*np.linalg.norm(lmid_laser_coordinate)))
print(yaw_degree_perpendicular_Pmid)
print(math.degrees(yaw_degree_perpendicular_Pmid))

0.22553221162218626
12.922043870202605
